# Classification

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import chi2
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_auc_score ,roc_curve,auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from collections import Counter
import numpy as np
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support,accuracy_score
from sklearn import tree
import xgboost as xgb

import warnings
warnings.simplefilter('ignore')

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
classification_file=pd.read_excel(r'classification_file.xlsx')

In [ ]:

classification_train = classification_file[[ 'count_at', 'anger', 'anticipation', 'disgust', 'fear',
       'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', 'pros_title_length_y','count_mentions', 'count_video', 'count_words',
       'count_uppercased', 'count_lowercased', 'count_punctuation',
       'total_shares','LABEL',
       'number_of_authors', 'polarity']]
y =  classification_train.LABEL
X =  classification_train.drop('LABEL',axis=1)
scaler=StandardScaler()
scaled_X = scaler.fit_transform(X)


In [ ]:
def classify(X,y,classifier):   
    kf = StratifiedKFold(n_splits=10,shuffle=True,random_state=100)
    acc_score=[]
    prec_score =[]
    f_score =[]
    reca_score =[]
    i=1
    for train_index,test_index in kf.split(X,y):
        #print('{} of KFold {}'.format(i,kf.n_splits))
        xtr,xvl = X[train_index],X[test_index]
        ytr,yvl = y[train_index],y[test_index]

        #model
        clfr = classifier
        clfr.fit(xtr,ytr)
        pred2= clfr.predict(xvl) 
        Accuracy= accuracy_score(yvl, pred2)
        #confusion matrix',confusion_matrix(yvl, pred))
        recall = metrics.recall_score(yvl, pred2)
        precision = metrics.precision_score(yvl, pred2)
        fscore=metrics.f1_score(yvl, pred2)
        #print(metrics.confusion_matrix(yvl,pred2))
        
        #print('confusion matrix',confusion_matrix(yvl, pred2))
        #print('precision_recall_fscore_support',precision_recall_fscore_support(yvl, pred2))
        acc_score.append(Accuracy)
        prec_score.append(precision)
        reca_score.append(recall)
        f_score.append(fscore)
        i+=1
    e = (sum(acc_score)/len(acc_score))
    f= (sum(prec_score)/len(prec_score))
    g=(sum(reca_score)/len(reca_score))
    h=(sum(f_score)/len(f_score))
    print(classifier,':')
    print('\033[1m'+'Average of all Precision :' +'\033[0m',f)
    print('\033[1m'+'Average of all Accuracy :' +'\033[0m',e)
    print('\033[1m'+'Average of all Recall :' +'\033[0m',g)
    print('\033[1m'+'Average of all F1-score :' +'\033[0m',h)
    return e,f,g,h
    


In [ ]:
def call_classify(X,y):    #our dataset is not imbalanced
    accuracy=[]
    avg_precision=[]
    recall=[]
    f_score=[]
    classifiers = [RandomForestClassifier(n_estimators=200,n_jobs=-1,random_state=100),
                   LogisticRegression(random_state=100),SVC(kernel='linear',random_state=100,probability=True),
                   ExtraTreesClassifier(n_estimators=100, random_state=100),GaussianNB(),
                   xgb.XGBClassifier(objective="binary:logistic", random_state=100)]
    for classifier in (classifiers):
            e,f,g,h = classify(X,y,classifier)
            accuracy.append(e*100)
            avg_precision.append(f*100)
            recall.append(g*100)
            f_score.append(h*100)
    return accuracy,f_score,avg_precision,recall
classif = ['Random Forest','Logistic_regression','SVM','Extratrees classifier','GaussianNB','XGBoost']        

In [ ]:
e1,f1,g1,h1=call_classify(scaled_X,y) 

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=100) :
Average of all Precision : 0.7900787005392269
Average of all Accuracy : 0.7630675526024364
Average of all Recall : 0.7255411255411256
Average of all F1-score : 0.753149480004273
LogisticRegression(random_state=100) :
Average of all Precision : 0.666146594146594
Average of all Accuracy : 0.6941860465116279
Average of all Recall : 0.7768398268398269
Average of all F1-score : 0.7156512848493041
SVC(kernel='linear', probability=True, random_state=100) :
Average of all Precision : 0.6389743124469511
Average of all Accuracy : 0.6847729789590256
Average of all Recall : 0.8582251082251083
Average of all F1-score : 0.7306188973016308
ExtraTreesClassifier(random_state=100) :
Average of all Precision : 0.7957382621865055
Average of all Accuracy : 0.7863787375415282
Average of all Recall : 0.7766233766233765
Average of all F1-score : 0.7835592214832277
GaussianNB() :
Average of all Precision : 0.5558453316244437
Average of all

In [ ]:
result = pd.DataFrame(
    {'Classifier': classif,
     'Accuracy': e1,
     'Precision': f1,
     'Recall': g1,
     'F1-score': h1,
    })

result

,Classifier,Accuracy,Precision,Recall,F1-score
0,Random Forest,76.306755,75.314948,79.007870,72.554113
1,Logistic_regression,69.418605,71.565128,66.614659,77.683983
2,SVM,68.477298,73.061890,63.897431,85.822511
3,Extratrees classifier,78.637874,78.355922,79.573826,77.662338
4,GaussianNB,58.754153,68.204117,55.584533,88.593074
5,XGBoost,75.348837,75.176799,75.984896,75.346320
